<a href="https://colab.research.google.com/github/FrankT990/Alpaca_ATB/blob/main/ALPCA_Debugger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install alpaca-trade-api
# !pip install alpaca
# !pip install requests
# !pip install pandas
import time
import json
import requests
import pandas as pd
import math
import time
from datetime import datetime, timedelta
import yfinance as yf

In [ ]:
# This code block holds supplemental functions that help with the
# functions below

# Gets expiration date 2 weeks ahead
def get_exp_date():
   return (datetime.now() + timedelta(days=14)).strftime('%Y-%m-%d')

# Gets current price of asset
# Takes in string of stock ticker
# (uses yahoo finance because I couldn't get Alpaca's historical
#  stock data to work for some reason)
def get_curr_price(tkr):
  stock = yf.Ticker(tkr)
  data = stock.history(period='1d')
  if not data.empty:
      recent_price = data['Close'].iloc[-1]
      return recent_price
  else:
      return -1

# Posts an order to alpaca
# side = 'buy' or 'sell
# symbol = asset symbol (can be stock ticker or options contract id)
# qty = quantity
# limit_price = limit_price
def post_order(side, symbol, qty, limit_price, headers):
  url = "https://paper-api.alpaca.markets/v2/orders"
  payload = {
      "side": side,
      "type": "limit",
      "time_in_force": "day",
      "symbol": symbol,
      "qty": qty,
      "limit_price": limit_price
  }
  # If successful response
  response = requests.post(url, json=payload, headers=headers)
  if (response.status_code == 200):
    return response
  else:
    return -1

def get_positions_tkrs(headers):
  url = "https://paper-api.alpaca.markets/v2/positions"
  response = requests.get(url, headers=headers)
  if (response.status_code == 200):
    resp_json = response.json()
    tkrs = []
    for position in resp_json:
      tkrs.append(position['symbol'])
    return tkrs
  else:
    return -1


In [ ]:
# This block is for functions that handle finding options contracts to trade

# Gets options contracts from alpaca
def get_options_contracts(tkr, headers, side):
  tkr = tkr
  side = side
  curr_price = get_curr_price(tkr)
  if (side=='put'):
    strike_price = curr_price * .98
  else:
    strike_price = curr_price * 1.02
  exp_date = get_exp_date()

  url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{tkr}?feed=indicative&limit=100&type={side}&strike_price_lte={strike_price}&expiration_date_lte={exp_date}"
  response = requests.get(url, headers=headers)
  if (response.status_code == 200):
    return response
  else:
    return -1

def find_put_options(tkr, headers):
  response = get_options_contracts(tkr, headers, 'put')
  if (response == -1):
    return -1, -1
  else:
    resp_json = response.json()
    data = resp_json['snapshots']
    contract_id = ""
    price = 0.0
    for contract in data:
      if (data[contract]['latestQuote']['ap'] > price):
        price = data[contract]['latestQuote']['ap']
        contract_id = contract
    return contract_id, price

def find_call_options(tkr, headers):
  response = get_options_contracts(tkr, headers, 'call')
  if (response == -1):
    return -1, -1
  else:
    resp_json = response.json()
    data = resp_json['snapshots']
    contract_id = ""
    price = 0.0
    contract_id = list(data.keys())[0]
    price = data[contract_id]['latestQuote']['ap']
    return contract_id, price

def get_open_orders(headers):
  url = "https://paper-api.alpaca.markets/v2/orders?status=open"
  response = requests.get(url, headers=headers)
  if (response.status_code == 200): return response.json()
  return -1


In [ ]:
def main():
  headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "APCA-API-KEY-ID": "Insert Alpaca key here",
    "APCA-API-SECRET-KEY": "Insert Alpaca secret key here"
  }

  # 0 = sell covered calls
  # 1 = sell cash secured puts
  tkr = 'NVDA'
  state = 0
  while True:
    positions = get_positions_tkrs(headers)
    open_orders = get_open_orders(headers)
    if (len(open_orders) == 0):
      continue
    if tkr in positions:
      # If we already have stock, we sell calls
      contract_id, price = find_call_options(tkr, headers)
      if (contract_id == -1 or price == -1):
        continue
      post_order('sell', contract_id, 1, price, headers)
      # ui_to_state_1
    else:
      # If we don't have any stock, we sell puts
      contract_id, price = find_put_options(tkr, headers)
      if (contract_id == -1 or price == -1):
        continue
      post_order('sell', contract_id, 1, price, headers)
      # ui_to_state_2

    time.sleep(3600)

if __name__ == '__main__':
    main()